In [1]:
import sys, os
from pathlib import Path
if "../src" not in sys.path:
    sys.path.append("../src")
    
import re
import pandas as pd
from collections import defaultdict

In [2]:
save_mode = True # if True, save generated data except for wiki_retrieved_data
post_edit_mode = True # if True, add author time from notes

In [3]:
from dietcoke.get_author import get_all_authors
authors_tier1, authors_tier12, authors_uni = get_all_authors(save_names_clean=save_mode)

No author time map ...


100%|██████████| 9/9 [00:22<00:00,  2.50s/it]


File saved: ../data/author_time/names_clean.txt


In [4]:
## print
from dietcoke.get_author import PAT_ANONYM
cnt_tier1 = sum([not re.match(PAT_ANONYM, n) for n in authors_tier1])
cnt_tier12 = sum([not re.match(PAT_ANONYM, n) for n in authors_tier12])

print('---\nCount of tier1:', cnt_tier1, '/', len(authors_tier1))
print('Coverage of tier1:', round(cnt_tier1 / len(authors_tier1), 2))

print('---\nCount of tier12:', cnt_tier12, '/', len(authors_tier12))
print('Coverage of tier12:', round(cnt_tier12 / len(authors_tier12), 2))

---
Count of tier1: 30 / 120
Coverage of tier1: 0.25
---
Count of tier12: 2772 / 2906
Coverage of tier12: 0.95


In [5]:
# from dietcoke.get_author import get_wiki_data
# get_wiki_data()

---

In [3]:
from dietcoke.get_author import get_wiki_author_time
get_wiki_author_time(save_wiki_author_time=save_mode)

No author time map ...
File read: ../data/author_time/wiki_retrieved_data.pkl
File saved: ../data/author_time/wiki_author_time.json
Count of retrieved data: 1182
Count of author time info: 855


In [3]:
from dietcoke.author import get_author_time_map
author_time = get_author_time_map()
print(len(author_time))

Mapping author time ...
File read: ../data/author_time/wiki_author_time.json
File read: author_time_map.json
File read: ../notes/post_edit.json
File read: ../data/author_time/wiki_author_time.json
File read: author_time_map.json
File read: ../notes/post_edit.json
943


In [4]:
sorted(author_time.items(), key=lambda x: x[1])[:10]

[('旧题管仲', [-725, -645]),
 ('管仲', [-725, -645]),
 ('左丘明', [-556, -451]),
 ('孙武', [-544, -470]),
 ('墨翟', [-468, -376]),
 ('列御寇', [-450, -375]),
 ('吴起', [-440, -381]),
 ('慎到', [-395, -315]),
 ('李斯', [-284, -208]),
 ('韩非', [-281, -233])]

In [5]:
from dietcoke import Corpus, dynaspan_lst, Author

for dynaspan in dynaspan_lst:
    print('-----\n', dynaspan)
    corpus = Corpus(dynaspan)
    corpus.read_corpus()

    c = 0
    for i, line in enumerate(corpus.corpus):
        rep_year = Author(line.author).rep_year
        if rep_year != -9999:
            c += 1
    print(c, '/', i)

-----
 先秦
7 / 40
-----
 漢
28 / 78
-----
 魏晉南北
46 / 78
-----
 唐五代十國
104 / 181
-----
 宋元
356 / 810
-----
 明
251 / 429
-----
 清
705 / 1280
-----
 民國
2 / 2


In [6]:
from dietcoke.author import get_author_time_map
group_authors = defaultdict(list)
for author, time in get_author_time_map(merge_data=False).items():
    group_authors[str(time)].append(author)

for life, author_lst in group_authors.items():
    if len(author_lst) >= 2:
        print(life, author_lst)

File read: ../data/author_time/wiki_author_time.json
[1079, 1154] ['汪藻', '许叔微']
[1420, 1474] ['岳正', '叶盛']
[1472, 1529] ['李梦阳', '王守仁']
[1512, 1565] ['胡宗宪', '韩叔阳']
[1558, 1639] ['郝敬', '陈继儒']
[1799, 1873] ['何绍基', '王柏心']
[1824, 1890] ['曾国荃', '黄彭年']
[1838, 1894] ['薛福成', '陆心源']
[1842, 1906] ['姚振宗', '王之春']
[1852, 1936] ['王树枏', '王树枬']


In [7]:
df = pd.DataFrame(list(group_authors.items()))
df.columns = ('life', 'author_lst')
df['mid_year'] = [Author(n[-1]).rep_year for n in df['author_lst']]
df.sort_values('mid_year')

if save_mode:
    df.to_csv('../data/author_time/author_timeline.csv')

df

,life,author_lst,mid_year
0,"[-450, -375]",[列御寇],-412
1,"[-250, -150]",[伏胜],-200
2,"[-179, -122]",[刘安],-151
3,"[-154, -93]",[东方朔],-124
4,"[32, 92]",[班固],62
...,...,...,...
836,"[1953, 2017]",[陈玉树],1985
837,[1962],[王希明],1962
838,[1973],[陈炜],1973
839,[1986],[许嵩],1986


In [8]:
# 莊述祖
# 王树枏/王树枬

---

In [9]:
from tqdm.auto import tqdm
from dietcoke import corpus_lst, Author
import pandas as pd

authors = []
profiles = []
for corpus in tqdm(corpus_lst()):
    corpus.read_corpus()
    dynaspan = corpus.dynaspan
    for line in corpus.corpus:
        line_authors = Author(line.author).name_norm
        for profile in Author(line.author).author_profile:
            profile['urn'] = line.urn
            profile['title'] = line.obj['title']
            profile['dynaspan'] = dynaspan
            profiles.append(profile)
            line_authors.remove(profile['name'])
        authors += [{'name': name,
        'urn': line.urn, 'title': line.obj['title'], 'dynaspan': dynaspan} for name in line_authors]

profile_df = pd.DataFrame(profiles).sort_values(['birth_year', 'name'])
author_df = pd.DataFrame(authors).sort_values('name')
profile_df = pd.merge(profile_df, author_df, how='outer')

if save_mode:
    profile_df.to_csv('author_profile.csv')

100%|██████████| 8/8 [00:22<00:00,  2.84s/it]
